In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df = pd.read_csv('tweets.csv')
df

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...
...,...,...,...
7915,7916,0,Live out loud #lol #liveoutloud #selfie #smile...
7916,7917,0,We would like to wish you an amazing day! Make...
7917,7918,0,Helping my lovely 90 year old neighbor with he...
7918,7919,0,Finally got my #smart #pocket #wifi stay conne...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7920 entries, 0 to 7919
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      7920 non-null   int64 
 1   label   7920 non-null   int64 
 2   tweet   7920 non-null   object
dtypes: int64(2), object(1)
memory usage: 185.8+ KB


In [ ]:
df['label'].value_counts()

0    5894
1    2026
Name: label, dtype: int64

In [ ]:
df.isna().sum()

id       0
label    0
tweet    0
dtype: int64

In [ ]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import re
import unidecode

In [ ]:
def clean_keywords(word):
    return re.sub(r'%20', ' ', word)
def to_lowercase(word):
    return word.lower()
def remove_accents(word):
    return unidecode.unidecode(word)
def remove_punctuation(word):
    return re.sub(r"[!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n -' ]"," ",word)

In [ ]:
def cleaning_URLs(word):
    return re.sub('((www.[^s]+)|(https?:\/\/.*?[\s+]))',' ',word)
def remove_mentions(word):
    return re.sub('@[\w]*',' ',word)

In [ ]:
df['cleaned_tweet'] = df['tweet'].apply(lambda x: to_lowercase(x))
df['cleaned_tweet'] = df['cleaned_tweet'].apply(lambda x: cleaning_URLs(x))
df['cleaned_tweet'] = df['cleaned_tweet'].apply(lambda x: remove_mentions(x))
df['cleaned_tweet'] = df['cleaned_tweet'].apply(lambda x: remove_accents(x))
df['cleaned_tweet'] = df['cleaned_tweet'].apply(lambda x: remove_punctuation(x))
df['cleaned_tweet'] = df['cleaned_tweet'].apply(lambda x: clean_keywords(x))


In [ ]:
df

,id,label,tweet,cleaned_tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...,fingerprint pregnancy test android apps ...
1,2,0,Finally a transparant silicon case ^^ Thanks t...,finally a transparant silicon case thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...,we love this would you go talk makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...,i m wired i know i m george i was made that wa...
4,5,1,What amazing service! Apple won't even talk to...,what amazing service apple won t even talk to...
...,...,...,...,...
7915,7916,0,Live out loud #lol #liveoutloud #selfie #smile...,live out loud lol liveoutloud selfie smile...
7916,7917,0,We would like to wish you an amazing day! Make...,we would like to wish you an amazing day make...
7917,7918,0,Helping my lovely 90 year old neighbor with he...,helping my lovely 90 year old neighbor with he...
7918,7919,0,Finally got my #smart #pocket #wifi stay conne...,finally got my smart pocket wifi stay conne...


In [ ]:
from gensim.models import Word2Vec as wtv

In [ ]:
preprocessed_text = df['cleaned_tweet'].apply(lambda x: x.split())

In [ ]:
preprocessed_text

0       [fingerprint, pregnancy, test, android, apps, ...
1       [finally, a, transparant, silicon, case, thank...
2       [we, love, this, would, you, go, talk, makemem...
3       [i, m, wired, i, know, i, m, george, i, was, m...
4       [what, amazing, service, apple, won, t, even, ...
                              ...                        
7915    [live, out, loud, lol, liveoutloud, selfie, sm...
7916    [we, would, like, to, wish, you, an, amazing, ...
7917    [helping, my, lovely, 90, year, old, neighbor,...
7918    [finally, got, my, smart, pocket, wifi, stay, ...
7919    [apple, barcelona, apple, store, bcn, barcelon...
Name: cleaned_tweet, Length: 7920, dtype: object

In [ ]:
cbow_w2v_model = wtv(preprocessed_text, vector_size=300, window=6, min_count=3, sg=0)
skgram_w2v_model = wtv(preprocessed_text, vector_size=300, window=6, min_count=3, sg=1)

In [ ]:
print("cbow vocalulary size:", len(cbow_w2v_model.wv.index_to_key))
print("skipgram vocalulary size:", len(skgram_w2v_model.wv.index_to_key))

cbow vocalulary size: 4217
skipgram vocalulary size: 4217


In [ ]:
skgram_w2v_model.wv.most_similar('fingerprint')

[('mix', 0.9943819642066956),
 ('cakes', 0.9936104416847229),
 ('lord', 0.993532121181488),
 ('applepay', 0.9934924840927124),
 ('town', 0.9927085041999817),
 ('walmart', 0.9925689101219177),
 ('unix', 0.9925651550292969),
 ('thin', 0.992548942565918),
 ('grip', 0.9925100803375244),
 ('states', 0.992387056350708)]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from gensim.models import KeyedVectors

In [ ]:
filename = '/content/drive/MyDrive/Colab Notebooks/GoogleNews-vectors-negative300.bin'

In [ ]:
google_w2v = KeyedVectors.load_word2vec_format(filename, binary=True)

In [ ]:
def get_embeddimng_w2v(doc_tokens, model, pretrained):
    embeddings = []
    if pretrained:
      for tok in doc_tokens:
        if tok in google_w2v:
          embeddings.append(google_w2v[tok])
    else:
      for tok in doc_tokens:
        if tok in model.wv.index_to_key:
            embeddings.append(model.wv.get_vector(tok))
    return np.mean(embeddings, axis=0)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score, accuracy_score

## Skipgram model

In [ ]:
X_x2v_model = preprocessed_text.apply(lambda x: get_embeddimng_w2v(x, skgram_w2v_model, False))
X_df_sg = pd.DataFrame(X_x2v_model.to_list())

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_df_sg, df['label'], test_size=0.15, random_state=134)

In [ ]:
log = LogisticRegression()
log.fit(x_train, y_train)

LogisticRegression()

In [ ]:
y_pred_log = log.predict(x_test)
print("accuracy score :",accuracy_score(y_test, y_pred_log))

accuracy score : 0.8863636363636364


In [ ]:
gnb = GaussianNB()
gnb.fit(x_train, y_train)

GaussianNB()

In [ ]:
y_pred_gnb = gnb.predict(x_test)
accuracy_score(y_test, y_pred_gnb)

0.851010101010101

# Cbow model

In [ ]:
X_x2v_model = preprocessed_text.apply(lambda x: get_embeddimng_w2v(x, cbow_w2v_model, False))
X_df_cbow = pd.DataFrame(X_x2v_model.to_list())

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_df_cbow, df['label'], test_size=0.15, random_state=134)

In [ ]:
log2 = LogisticRegression()
log2.fit(x_train, y_train)

LogisticRegression()

In [ ]:
y_pred_log2 = log2.predict(x_test)
print("accuracy score :",accuracy_score(y_test, y_pred_log2))

accuracy score : 0.8686868686868687


In [ ]:
gnb2 = GaussianNB()
gnb2.fit(x_train, y_train)

GaussianNB()

In [ ]:
y_pred_gnb2 = gnb2.predict(x_test)
accuracy_score(y_test, y_pred_gnb2)

0.8164983164983165

# Google Pretrained model

In [ ]:
X_x2v_model = preprocessed_text.apply(lambda x: get_embeddimng_w2v(x, None, True))

In [ ]:
X_df_google = pd.DataFrame(X_x2v_model.to_list())

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_df_google, df['label'], test_size=0.15, random_state=134)

In [ ]:
log3 = LogisticRegression()
log3.fit(x_train, y_train)

LogisticRegression()

In [ ]:
y_pred_log3 = log3.predict(x_test)
print("accuracy score :",accuracy_score(y_test, y_pred_log3))

accuracy score : 0.8804713804713805


In [ ]:
gnb3 = GaussianNB()
gnb3.fit(x_train, y_train)

GaussianNB()

In [ ]:
y_pred_gnb3 = gnb3.predict(x_test)
accuracy_score(y_test, y_pred_gnb3)

0.8114478114478114